In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Flatten, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from shared import read_dataset

2023-12-05 21:07:12.510679: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 21:07:12.541211: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 21:07:12.541244: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 21:07:12.541262: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 21:07:12.546639: I tensorflow/core/platform/cpu_feature_g

Data:<br>
dall'SP500 prendiamo `Open`, `Close`, `High`, `Low`, `Volume`

Target:
- `1` if the close price difference is over 1.5%
- `0` otherwise 

In [9]:
start = "2015-01-05"
end = "2017-12-29"

SP500 = read_dataset('../data/^GSPC.csv', start, end)
SP500['pct change'] = SP500['Close'].pct_change()
SP500.dropna(inplace=True)

SP500['target'] = SP500['pct change'].apply(lambda x: 1 if x > 0 else 0)
SP500 = SP500.drop('Adj Close', axis=1)

print(SP500.shape)

SP500.head()

(753, 7)


,Open,High,Low,Close,Volume,pct change,target
Date,,,,,,,
2015-01-06,2022.150024,2030.250000,1992.439941,2002.609985,4460110000,-0.008893,0
2015-01-07,2005.550049,2029.609985,2005.550049,2025.900024,3805480000,0.011630,1
2015-01-08,2030.609985,2064.080078,2030.609985,2062.139893,3934010000,0.017888,1
2015-01-09,2063.449951,2064.429932,2038.329956,2044.810059,3364140000,-0.008404,0
2015-01-12,2046.130005,2049.300049,2022.579956,2028.260010,3456460000,-0.008094,0


In [10]:
data = np.array(SP500.values)
target = np.array(SP500['target'].values)

# scaler_data = MinMaxScaler()
# scaler_data.fit(data)
# data = scaler_data.transform(data)

train_size = int(len(data) * 0.80)

n_features = data.shape[-1]
n_timesteps = 20
def create_sequences(data, target, n_timesteps):
    X, y = [], []
    for i in range(len(data) - n_timesteps):
        X.append(data[i:i+n_timesteps])
        y.append(target[i+n_timesteps])
    return np.array(X), np.array(y)

data_seq, target_seq = create_sequences(data, target, n_timesteps)

x_train, x_test = np.array(data_seq[:train_size]), np.array(data_seq[train_size:])
y_train_class, y_test_class = np.array(target_seq[:train_size]), np.array(target_seq[train_size:])

y_train = np.eye(2)[y_train_class]
y_test = np.eye(2)[y_test_class]

print(f'train_data: {x_train.shape} train_target: {y_train.shape} | y class: {y_train_class.shape}')
print(f'test_data: {x_test.shape} test_target: {y_test.shape} | y class: {y_test_class.shape}')

train_data: (602, 20, 7) train_target: (602, 2) | y class: (602,)
test_data: (131, 20, 7) test_target: (131, 2) | y class: (131,)


In [11]:
# TEST
index = 16
column = 5
print(f'Train [{index}]:')
print(x_train[index][:,column])
print(y_train[index])
print(y_train_class[index])
# print(w_[index])

Train [16]:
[ 9.53468529e-03 -1.29919653e-02  1.29624637e-02  1.44394953e-02
 -4.15604621e-03  1.02914067e-02 -3.41817231e-03 -4.24719436e-03
  1.06755610e-02 -2.90335887e-05  9.64450643e-03  4.07473849e-03
  1.59757446e-03 -3.14309218e-04 -1.06205733e-03  6.12653363e-03
 -3.03339195e-04  2.75877089e-03 -7.65723648e-04 -1.47602823e-03]
[1. 0.]
0


In [27]:
def build_model(input_shape, l2_value):
    kernel_size = 3
    pool_size = 1
    l2_ = l2(l2_value)
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv1D(32, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(64, kernel_size, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2_, bias_regularizer=l2_))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(128, kernel_size, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2_, bias_regularizer=l2_))
    # model.add(MaxPooling1D(pool_size=pool_size))
    # model.add(Conv1D(256, kernel_size, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2_, bias_regularizer=l2_))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(256, kernel_size, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2_, bias_regularizer=l2_))

    model.add(Flatten())

    # model.add(Dense(256, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2_, bias_regularizer=l2_))
    model.add(Dense(256, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2_, bias_regularizer=l2_))

    model.add(Dense(2, activation='softmax'))

    return model

In [28]:
batch_size = 1000
epochs = 1000
patience = 20
l2_value = 0.006
input_shape = (n_timesteps, n_features)

model = build_model(input_shape, l2_value)
model.compile(loss='binary_crossentropy',
            optimizer=Adam(),
            metrics=[Recall(), 'accuracy', Precision()])

history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.2,
                    callbacks=[EarlyStopping(
                        monitor='val_loss',
                        patience=patience,
                        verbose=2,
                        mode='min',
                        restore_best_weights=True,
                    )])

Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 18009474.0000 - recall_7: 0.5052 - accuracy: 0.5052 - precision_7: 0.5052 - val_loss: 574615296.0000 - val_recall_7: 0.5289 - val_accuracy: 0.5289 - val_precision_7: 0.5289
Epoch 2/1000
1/1 [==============================] - 0s 40ms/step - loss: 654315008.0000 - recall_7: 0.4990 - accuracy: 0.4990 - precision_7: 0.4990 - val_loss: 262238688.0000 - val_recall_7: 0.5289 - val_accuracy: 0.5289 - val_precision_7: 0.5289
Epoch 3/1000
1/1 [==============================] - 0s 40ms/step - loss: 298629792.0000 - recall_7: 0.4990 - accuracy: 0.4990 - precision_7: 0.4990 - val_loss: 126784776.0000 - val_recall_7: 0.5289 - val_accuracy: 0.5289 - val_precision_7: 0.5289
Epoch 4/1000
1/1 [==============================] - 0s 41ms/step - loss: 138607584.0000 - recall_7: 0.4990 - accuracy: 0.4990 - precision_7: 0.4990 - val_loss: 105793000.0000 - val_recall_7: 0.4711 - val_accuracy: 0.4711 - val_precision_7: 0.4711
Epoch 5/1000
1/

In [29]:
y_pred = model.predict(x_test)
y_pred_classes = np.array([1 if entry[0] < entry[1] else 0 for entry in y_pred])
y_true = y_test_class

conf_matrix = confusion_matrix(y_true, y_pred_classes)

accuracy = accuracy_score(y_true, y_pred_classes)
report = classification_report(y_true, y_pred_classes)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)


5/5 [==============================] - 0s 1ms/step
Accuracy: 0.5114503816793893
Confusion Matrix:
 [[24 28]
 [36 43]]
Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.46      0.43        52
           1       0.61      0.54      0.57        79

    accuracy                           0.51       131
   macro avg       0.50      0.50      0.50       131
weighted avg       0.52      0.51      0.52       131

